In [ ]:
import os
import skimage
import fnmatch

import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
import re
import pickle


def resize_2d_nonan(array,factor):
    """
    intial author: damo_ma
    """
    xsize, ysize = array.shape

    if isinstance(factor,int):
        factor_x = factor
        factor_y = factor
    elif isinstance(factor,tuple):
        factor_x , factor_y = factor[0], factor[1]
    else:
        raise NameError('Factor must be a tuple (x,y) or an integer')

    if not (xsize %factor_x == 0 or ysize % factor_y == 0) :
        raise NameError('Factors must be intger multiple of array shape')

    new_xsize, new_ysize = xsize//factor_x, ysize//factor_y

    new_array = np.empty([new_xsize, new_ysize])
    new_array[:] = np.nan # this saves us an assignment in the loop below

    # submatrix indexes : is the average box on the original matrix
    subrow, subcol  = np.indices((factor_x, factor_y))

     # new matrix indexs
    row, col  = np.indices((new_xsize, new_ysize))

    for i, j, ind in zip(row.reshape(-1), col.reshape(-1),range(row.size)) :
        # define the small sub_matrix as view of input matrix subset
        sub_matrix = array[subrow+i*factor_x,subcol+j*factor_y]
        # modified from any(a) and all(a) to a.any() and a.all()
        # see https://stackoverflow.com/a/10063039/1435167
        if (np.isnan(sub_matrix)).sum() < (factor_x*factor_y)/2.0 + (np.random.rand() -0.5): # if we haven't all NaN
            if (np.isnan(sub_matrix)).any(): # if we haven no NaN at all
                (new_array.reshape(-1))[ind] = np.nanmean(sub_matrix)
            else: # if we haven some NaN
                (new_array.reshape(-1))[ind] = np.mean(sub_matrix)
        # the case assign NaN if we have all NaN is missing due 
        # to the standard values of new_array

    return new_array


def load_pfm(fname):
    color = None
    width = None
    height = None
    scale = None
    endian = None

    file = open(fname,'r',encoding='iso-8859-1')
    header = file.readline().rstrip()
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline())
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().rstrip())
    if scale < 0: # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>' # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)
    return np.flipud(np.reshape(data, shape)), scale
def load_calibtxt(fname):
    file = open(fname,'rt',encoding='iso-8859-1')
    return {l.split('=')[0]:' '.join(l.split('=')[1:]).rstrip() for l in [line for line in file]}
def cf(num1,num2):
    n=[]
    for i in range(1, min(num1, num2)+1): 
        if num1%i==num2%i==0: 
            n.append(i)
    return n


In [ ]:
in_fold = 'data/middlebury/middle_custom/'
out_fold = 'data/middlebury/mid_out2'
target_size = 215000

In [ ]:
if not os.path.exists(out_fold):
    os.mkdir(out_fold)

In [ ]:
target_dirs = sorted([_ for _ in os.listdir(in_fold) if os.path.exists(os.path.join(in_fold,_,'im0.png'))])

In [ ]:
for tdir in target_dirs:
    #if 'perfect' in tdir: continue

    im0 = skimage.io.imread(os.path.join(in_fold,tdir,'im0.png'))
    im1 = skimage.io.imread(os.path.join(in_fold,tdir,'im1.png'))
    scales = [(i,np.prod(im0.shape[:2])/(i*i)) for i in cf(*im0.shape[:2])]
    scales = sorted([(abs(np.log(target_size/_[1])),_[0],_[1]) for _ in scales])
    scale = scales[0][1]
    im0 = skimage.transform.downscale_local_mean(im0,(scale,scale,1))
    im1 = skimage.transform.downscale_local_mean(im1,(scale,scale,1))
    gt,s = load_pfm(os.path.join(in_fold,tdir,'disp0.pfm'))
    gt2 = resize_2d_nonan(gt,scale)/scale
    max_disp = int(load_calibtxt(os.path.join(in_fold,tdir,'calib.txt'))['ndisp'])
    max_disp = int(load_calibtxt(os.path.join(in_fold,tdir,'calib.txt'))['ndisp'])
    max_disp = int(np.ceil(max_disp/scale))
    max_disp= max(16,max_disp)
    max_disp = (max_disp//16+1)*16 if max_disp//16 != max_disp/16 else max_disp
    print(tdir,scale,max_disp)
    
    gt_d2,s_d2 = load_pfm(os.path.join(in_fold,tdir,'disp1.pfm'))
    gt2_d2 = resize_2d_nonan(gt_d2,scale)/scale

    new_img = []
    fine_thresh =1
    for row1,row2 in zip(gt2,gt2_d2):
        s_d = np.arange(len(row1))-row1
        v2 = np.round(np.clip(s_d,0,len(row1)-1)).astype(int)
        vdiff = np.nan_to_num(abs(row1-row2[v2]))
        new_img.append(vdiff > fine_thresh)

    
    out_d = {
        'im0': np.round(im0).astype(np.uint8),
        'im1': np.round(im1).astype(np.uint8),
        'max_disp': max_disp,
        'gt': gt2,
        'mask': np.array(new_img)
    }
    
    with open(os.path.join(out_fold,"{}.pkl".format(tdir)), "wb") as f: 
        pickle.dump(out_d, f)

In [ ]:
cols = ['P1', 'P2', 'blockSize', 'disp12MaxDiff', 'preFilterCap', 'speckleRange', 'speckleWindowSize', 'uniquenessRatio']
x0 = np.log(np.array([216, 864, 5, 40, 63, 2, 0.1, 15]))


In [ ]:
import cv2
errs = []
disps = []
for tdir in target_dirs:
    
    with open(os.path.join(out_fold,"{}.pkl".format(tdir)), "rb") as f: 
        out_d = pickle.load(f)
    
    params = x0
    p = {
        'blockSize': 5,
        'P1':8 * 3 * 3 ** 2,
        'P2':32 * 3 * 3 ** 2,
        'disp12MaxDiff': 40,
        'uniquenessRatio': 15,
        'speckleWindowSize': 0,
        'speckleRange': 2,
        'preFilterCap': 63,
    }

    p['minDisparity'] = 0
    p['numDisparities'] = out_d['max_disp']
    p['mode'] = cv2.STEREO_SGBM_MODE_SGBM

    for c,v in zip(cols,params):
        p[c] = int(round(np.exp(v)))

    left_matcher = cv2.StereoSGBM_create(**p)

    max_disp = out_d['max_disp']
    imL = np.pad(out_d['im0'], [(0,0), (max_disp,0), (0,0)],mode='constant')
    imR = np.pad(out_d['im1'], [(0,0), (max_disp,0), (0,0)],mode='constant')
    displ = left_matcher.compute(imL, imR)[:,max_disp:]
    result = (displ).astype(np.float32)/16
    max_error = 3

    err_vec = np.nan_to_num(abs(result-out_d['gt']))
    err_vec[err_vec > max_error] = max_error
    err_vec[displ == -1] = max_error
    err = np.mean(err_vec)
    errs.append(err)
    disps.append(max_disp)
    
np.mean(errs),np.std(errs)

In [ ]:
a  = len(errs)//2
b = len(errs)-a
s = [True]*a + [False]*b
import random
combos = [] 
for i in range(10000000): # 10000000
    combos.append(tuple(sorted(s, key=lambda k: random.random())))

In [ ]:
import itertools
e_arr = np.array(errs)
d_arr = np.array(disps)
err_val ={}
b_val ={}

for combo in combos:    
    c_arr = np.array(combo)
    big_err = [abs(f(e_arr[c_arr])-f(e_arr[~c_arr])) for f in [np.mean,np.std]] + \
              [abs(f(d_arr[c_arr])-f(d_arr[~c_arr])) for f in [np.mean,np.std]]
    err_val[combo] = np.prod(big_err)
    b_val[combo] = big_err
ideal_res = sorted([(v,np.where(k)[0],k) for k,v in err_val.items()])
ideal_res_tmp = sorted([(v,np.where(k)[0]) for k,v in err_val.items()])

ideal_res_tmp[:3],b_val[ideal_res[0][2]]

In [ ]:
big_err